# Advent of Code

## 2019-012-018
## 2019 018

https://adventofcode.com/2019/day/18

In [1]:
from collections import deque

# Directions for movement (up, down, left, right)
DIRS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

def parse_input(file_path):
    with open(file_path, 'r') as file:
        maze = [list(line.strip()) for line in file.readlines()]
    return maze

def is_valid(maze, x, y):
    return 0 <= x < len(maze) and 0 <= y < len(maze[0])

def bfs(maze, start_x, start_y):
    # Map for holding the shortest path for each state
    n = len(maze)
    m = len(maze[0])
    total_keys = sum(1 for row in maze for cell in row if cell.islower())
    key_positions = {chr(k): [] for k in range(ord('a'), ord('z')+1)}

    # Collect all the key positions
    for i in range(n):
        for j in range(m):
            if maze[i][j].islower():
                key_positions[maze[i][j]].append((i, j))
                
    # BFS queue, containing tuples of (x, y, collected_keys, steps_taken)
    start_state = (start_x, start_y, 0, 0) # The start does not have any keys
    queue = deque([start_state])
    
    # Set to track visited states (x, y, collected_keys_bitmask)
    visited = set([start_state])
    
    while queue:
        x, y, collected_keys, steps = queue.popleft()

        # If we have collected all the keys, return the number of steps taken
        if collected_keys == (1 << total_keys) - 1:
            return steps
        
        # Explore all the adjacent positions
        for dx, dy in DIRS:
            nx, ny = x + dx, y + dy
            
            if not is_valid(maze, nx, ny):
                continue
            
            cell = maze[nx][ny]
            new_collected_keys = collected_keys
            new_steps = steps + 1
            
            # If it's a door, we can pass if we have the key
            if cell.isupper() and not (collected_keys & (1 << (ord(cell.lower()) - ord('a')))):
                continue
            
            # If it's a key, we collect it and update the collected_keys bitmask
            if cell.islower():
                new_collected_keys |= (1 << (ord(cell) - ord('a')))
            
            # Avoid revisiting the same position with the same set of keys
            if (nx, ny, new_collected_keys) not in visited:
                visited.add((nx, ny, new_collected_keys))
                queue.append((nx, ny, new_collected_keys, new_steps))

# Driver function to call and process the maze
def main():
    maze = parse_input('input.txt')
    
    # Find the starting position (@)
    start_x, start_y = None, None
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == '@':
                start_x, start_y = i, j
                break
        if start_x is not None:
            break

    # Perform BFS to find the shortest path
    result = bfs(maze, start_x, start_y)
    print(f"The shortest path to collect all the keys is {result} steps.")

if __name__ == "__main__":
    main()

MemoryError: 

In [2]:
from collections import deque, defaultdict

def parse_map(input_file):
    with open(input_file, 'r') as file:
        grid = file.read().splitlines()
    keys = {}
    doors = {}
    start = None
    for y, line in enumerate(grid):
        for x, char in enumerate(line):
            if char == '@':
                start = (x, y)
            elif 'a' <= char <= 'z':
                keys[char] = (x, y)
            elif 'A' <= char <= 'Z':
                doors[char] = (x, y)
    return grid, keys, doors, start

def get_neighbors(x, y):
    return [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]

def shortest_path_to_collect_keys(grid, keys, doors, start):
    all_keys = frozenset(keys.keys())
    queue = deque([(start, frozenset(), 0)])  # Position, keys collected, steps
    visited = set()

    while queue:
        position, collected_keys, steps = queue.popleft()
        if (position, collected_keys) in visited:
            continue
        visited.add((position, collected_keys))

        # Check if all keys are collected
        if collected_keys == all_keys:
            return steps

        x, y = position
        for nx, ny in get_neighbors(x, y):
            if grid[ny][nx] == '#':  # Wall
                continue

            cell = grid[ny][nx]
            if 'A' <= cell <= 'Z' and cell.lower() not in collected_keys:
                continue  # Locked door

            if 'a' <= cell <= 'z':  # Key
                new_keys = collected_keys | frozenset([cell])
            else:
                new_keys = collected_keys

            queue.append(((nx, ny), new_keys, steps + 1))

    return -1  # No solution

# Main execution
input_file = 'input.txt'  # Replace with the uploaded file path if needed
grid, keys, doors, start = parse_map(input_file)
result = shortest_path_to_collect_keys(grid, keys, doors, start)
print("Shortest path to collect all keys:", result)

Shortest path to collect all keys: 3832


In [ ]:
import time
from heapq import heappop, heappush
from collections import defaultdict, deque


def parse_map(input_file):
    with open(input_file, 'r') as file:
        grid = file.read().splitlines()

    # Update the map for 4 robots
    for y, line in enumerate(grid):
        if '@' in line:
            x = line.index('@')
            grid[y - 1] = grid[y - 1][:x - 1] + '@#@' + grid[y - 1][x + 2:]
            grid[y] = grid[y][:x - 1] + '###' + grid[y][x + 2:]
            grid[y + 1] = grid[y + 1][:x - 1] + '@#@' + grid[y + 1][x + 2:]
            break

    robots = [(x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if char == '@']
    keys = {char: (x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if 'a' <= char <= 'z'}
    doors = {char: (x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if 'A' <= char <= 'Z'}
    return grid, robots, keys, doors


def get_neighbors(x, y):
    return [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]


def precompute_distances(grid, keys, robots):
    """Precompute distances between all keys and from robots to keys."""
    def bfs(start):
        queue = deque([(start, 0)])
        distances = {}
        visited = set()
        while queue:
            (x, y), dist = queue.popleft()
            if (x, y) in visited:
                continue
            visited.add((x, y))
            cell = grid[y][x]
            if cell in keys:
                distances[cell] = dist
            for nx, ny in get_neighbors(x, y):
                if grid[ny][nx] != '#' and (nx, ny) not in visited:
                    queue.append(((nx, ny), dist + 1))
        return distances

    key_distances = {key: bfs(pos) for key, pos in keys.items()}
    robot_distances = [bfs(robot) for robot in robots]
    return key_distances, robot_distances


def heuristic(remaining_keys, current_positions, keys, key_distances):
    """Estimate the minimum cost to collect all remaining keys."""
    h = 0
    for robot in current_positions:
        distances = [key_distances.get(key, {}).get(robot, float('inf')) for key in remaining_keys]
        h += min(distances) if distances else 0
    return h


def shortest_path_multi_robot(grid, robots, keys, doors):
    all_keys = frozenset(keys.keys())
    key_distances, robot_distances = precompute_distances(grid, keys, robots)
    queue = [(0, 0, tuple(robots), frozenset())]  # Total cost, steps, robot positions, collected keys
    visited = set()

    while queue:
        total_cost, steps, positions, collected_keys = heappop(queue)

        # Check if all keys are collected
        if collected_keys == all_keys:
            return steps

        # State memoization
        state = (positions, collected_keys)
        if state in visited:
            continue
        visited.add(state)

        for i, (x, y) in enumerate(positions):
            for nx, ny in get_neighbors(x, y):
                cell = grid[ny][nx]

                # Skip walls
                if cell == '#':
                    continue

                # Skip doors if the corresponding key hasn't been collected
                if 'A' <= cell <= 'Z' and cell.lower() not in collected_keys:
                    continue

                # Update keys if a new key is collected
                new_keys = collected_keys
                if 'a' <= cell <= 'z' and cell not in collected_keys:
                    new_keys = collected_keys | {cell}

                new_positions = list(positions)
                new_positions[i] = (nx, ny)
                h = heuristic(all_keys - new_keys, new_positions, keys, key_distances)
                heappush(queue, (steps + 1 + h, steps + 1, tuple(new_positions), new_keys))

    return -1  # No solution


# Main execution with nanosecond time logging
if __name__ == "__main__":
    input_file = 'input.txt'  # Replace with the uploaded file path if needed

    start_time = time.time_ns()  # Start timing in nanoseconds
    grid, robots, keys, doors = parse_map(input_file)
    result = shortest_path_multi_robot(grid, robots, keys, doors)
    end_time = time.time_ns()  # End timing in nanoseconds

    execution_time_seconds = (end_time - start_time) / 1_000_000_000  # Convert to seconds
    print("Fewest steps to collect all keys:", result)
    print(f"Execution time: {execution_time_seconds:.9f} s")

In [1]:
import time
from heapq import heappop, heappush
from collections import defaultdict, deque


def parse_map(input_file):
    with open(input_file, 'r') as file:
        grid = file.read().splitlines()

    # Update the map for 4 robots
    for y, line in enumerate(grid):
        if '@' in line:
            x = line.index('@')
            grid[y - 1] = grid[y - 1][:x - 1] + '@#@' + grid[y - 1][x + 2:]
            grid[y] = grid[y][:x - 1] + '###' + grid[y][x + 2:]
            grid[y + 1] = grid[y + 1][:x - 1] + '@#@' + grid[y + 1][x + 2:]
            break

    robots = [(x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if char == '@']
    keys = {char: (x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if 'a' <= char <= 'z'}
    doors = {char: (x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if 'A' <= char <= 'Z'}
    return grid, robots, keys, doors


def get_neighbors(x, y):
    return [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]


def shortest_path_multi_robot(grid, robots, keys, doors):
    all_keys = frozenset(keys.keys())
    queue = [(0, 0, tuple(robots), frozenset())]  # Total cost, steps, robot positions, collected keys
    visited = set()

    states_processed = 0  # Track the number of states processed
    progress_interval = 10000  # Print progress every 10,000 states
    last_update_time = time.time()
    start_time = last_update_time

    while queue:
        total_cost, steps, positions, collected_keys = heappop(queue)

        # Check if all keys are collected
        if collected_keys == all_keys:
            return steps

        # State memoization
        state = (positions, collected_keys)
        if state in visited:
            continue
        visited.add(state)

        states_processed += 1

        # Print progress every `progress_interval` states
        if states_processed % progress_interval == 0:
            current_time = time.time()
            elapsed_since_last = current_time - last_update_time
            total_elapsed = current_time - start_time
            print(
                f"Progress: {states_processed} states processed | "
                f"Time since last update: {elapsed_since_last:.2f}s | "
                f"Total time: {total_elapsed:.2f}s"
            )
            last_update_time = current_time

        for i, (x, y) in enumerate(positions):
            for nx, ny in get_neighbors(x, y):
                cell = grid[ny][nx]

                # Skip walls
                if cell == '#':
                    continue

                # Skip doors if the corresponding key hasn't been collected
                if 'A' <= cell <= 'Z' and cell.lower() not in collected_keys:
                    continue

                # Update keys if a new key is collected
                new_keys = collected_keys
                if 'a' <= cell <= 'z' and cell not in collected_keys:
                    new_keys = collected_keys | {cell}

                new_positions = list(positions)
                new_positions[i] = (nx, ny)
                heappush(queue, (steps + 1, steps + 1, tuple(new_positions), new_keys))

    return -1  # No solution


# Main execution with nanosecond time logging
if __name__ == "__main__":
    input_file = 'input.txt'  # Replace with the uploaded file path if needed

    start_time = time.time_ns()  # Start timing in nanoseconds
    grid, robots, keys, doors = parse_map(input_file)
    result = shortest_path_multi_robot(grid, robots, keys, doors)
    end_time = time.time_ns()  # End timing in nanoseconds

    execution_time_seconds = (end_time - start_time) / 1_000_000_000  # Convert to seconds
    print("Fewest steps to collect all keys:", result)
    print(f"Execution time: {execution_time_seconds:.9f} s")

Progress: 10000 states processed | Time since last update: 0.39s | Total time: 0.39s
Progress: 20000 states processed | Time since last update: 0.48s | Total time: 0.87s
Progress: 30000 states processed | Time since last update: 0.62s | Total time: 1.48s
Progress: 40000 states processed | Time since last update: 0.52s | Total time: 2.00s
Progress: 50000 states processed | Time since last update: 0.69s | Total time: 2.69s
Progress: 60000 states processed | Time since last update: 0.97s | Total time: 3.66s
Progress: 70000 states processed | Time since last update: 0.72s | Total time: 4.38s
Progress: 80000 states processed | Time since last update: 0.63s | Total time: 5.02s
Progress: 90000 states processed | Time since last update: 0.71s | Total time: 5.73s
Progress: 100000 states processed | Time since last update: 0.80s | Total time: 6.52s
Progress: 110000 states processed | Time since last update: 0.76s | Total time: 7.28s
Progress: 120000 states processed | Time since last update: 0.6

KeyboardInterrupt: 

In [ ]:
import time
from heapq import heappop, heappush
from collections import deque, defaultdict


def parse_map(input_file):
    with open(input_file, 'r') as file:
        grid = file.read().splitlines()

    # Update the map for 4 robots
    for y, line in enumerate(grid):
        if '@' in line:
            x = line.index('@')
            grid[y - 1] = grid[y - 1][:x - 1] + '@#@' + grid[y - 1][x + 2:]
            grid[y] = grid[y][:x - 1] + '###' + grid[y][x + 2:]
            grid[y + 1] = grid[y + 1][:x - 1] + '@#@' + grid[y + 1][x + 2:]
            break

    robots = [(x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if char == '@']
    keys = {char: (x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if 'a' <= char <= 'z'}
    doors = {char: (x, y) for y, line in enumerate(grid) for x, char in enumerate(line) if 'A' <= char <= 'Z'}
    return grid, robots, keys, doors


def get_neighbors(x, y):
    return [(x + dx, y + dy) for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]]


def precompute_distances(grid, positions):
    """
    Precompute distances between all keys and starting positions using BFS.
    """
    distances = {}
    for start_key, (sx, sy) in positions.items():
        queue = deque([(sx, sy, 0)])
        visited = set()
        while queue:
            x, y, dist = queue.popleft()
            if (x, y) in visited:
                continue
            visited.add((x, y))

            cell = grid[y][x]
            if cell in positions and cell != start_key:
                distances[(start_key, cell)] = dist

            for nx, ny in get_neighbors(x, y):
                if grid[ny][nx] != '#' and (nx, ny) not in visited:
                    queue.append((nx, ny, dist + 1))
    return distances


def shortest_path_multi_robot(grid, robots, keys, doors):
    """
    Optimized solution for multi-robot key collection with logging.
    """
    all_keys = (1 << len(keys)) - 1  # Bitmask representing all keys collected
    positions = {**keys, **{chr(ord('0') + i): pos for i, pos in enumerate(robots)}}
    distances = precompute_distances(grid, positions)
    queue = [(0, tuple(robots), 0)]  # (steps, positions, keys_collected)
    visited = {}

    # Logging variables
    states_processed = 0
    last_log_time = time.time_ns()
    start_time = time.time_ns()

    while queue:
        steps, current_positions, keys_collected = heappop(queue)
        states_processed += 1

        # Periodic logging
        if states_processed % 10_000 == 0:
            current_time = time.time_ns()
            elapsed_time_since_last_log = (current_time - last_log_time) / 1_000_000_000
            total_elapsed_time = (current_time - start_time) / 1_000_000_000
            print(
                f"States processed: {states_processed}, "
                f"Time since last log: {elapsed_time_since_last_log:.3f} s, "
                f"Total elapsed time: {total_elapsed_time:.3f} s"
            )
            last_log_time = current_time

        # If all keys are collected
        if keys_collected == all_keys:
            return steps

        state = (current_positions, keys_collected)
        if state in visited and visited[state] <= steps:
            continue
        visited[state] = steps

        for i, (x, y) in enumerate(current_positions):
            for key, (kx, ky) in keys.items():
                key_bit = 1 << (ord(key) - ord('a'))
                if keys_collected & key_bit:  # Skip if already collected
                    continue

                # Check path to the key
                required_steps = distances.get((chr(ord('0') + i), key), float('inf'))
                if required_steps == float('inf'):
                    continue

                # Check doors blocking the path
                for door, (dx, dy) in doors.items():
                    door_bit = 1 << (ord(door.lower()) - ord('a'))
                    if keys_collected & door_bit == 0 and (dx, dy) in distances:
                        continue

                # Move robot to key position
                new_positions = list(current_positions)
                new_positions[i] = (kx, ky)
                heappush(queue, (steps + required_steps, tuple(new_positions), keys_collected | key_bit))

    return -1  # No solution


# Main execution with nanosecond time logging
if __name__ == "__main__":
    input_file = 'input.txt'  # Replace with the uploaded file path if needed

    start_time = time.time_ns()  # Start timing in nanoseconds
    grid, robots, keys, doors = parse_map(input_file)
    result = shortest_path_multi_robot(grid, robots, keys, doors)
    end_time = time.time_ns()  # End timing in nanoseconds

    execution_time_seconds = (end_time - start_time) / 1_000_000_000  # Convert to seconds
    print("Fewest steps to collect all keys:", result)
    print(f"Execution time: {execution_time_seconds:.9f} s")